## Anime Recommendation System

In [61]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error

In [62]:
anime = pd.read_csv(r'C:\Users\visha\Desktop\anime.csv', encoding='ISO-8859-1')

anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


To visualize top rated Animations

In [63]:
anime.groupby('name')['rating'].mean().sort_values(ascending=False).head(10)

name
Taka no Tsume 8: Yoshida-kun no X-Files    10.00
Spoon-hime no Swing Kitchen                 9.60
Mogura no Motoro                            9.50
Kimi no Na wa.                              9.37
Kahei no Umi                                9.33
Fullmetal Alchemist: Brotherhood            9.26
Gintama°                                    9.25
Yakusoku: Africa Mizu to Midori             9.25
Steins;Gate                                 9.17
Gintama&#039;                               9.16
Name: rating, dtype: float64

To visualize most watched Animations

In [64]:
anime.groupby('name')['rating'].count().sort_values(ascending=False).head(10)

name
Shi Wan Ge Leng Xiaohua                2
Saru Kani Gassen                       2
Osomatsu-kun (1988)                    1
Oshiri Kajiri Mushi (TV)               1
Oshiri Kajiri Mushi (TV) 2nd Season    1
Oshiri Kajiri Mushi (TV) 3rd Season    1
Oshiri Kajiri Mushi (TV) 4th Season    1
Oshiruko                               1
Oshizuka ni                            1
Osiris no Tenbin                       1
Name: rating, dtype: int64

# Data Preprocessing : 

In [65]:
anime.loc[(anime["genre"]=="Hentai") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["type"] == "Movie") & (anime["episodes"] == "Unknown")] = "1"

In [66]:
known_animes = {"Detective Conan":854,"Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25, "Dragon Ball Super":86,"Naruto Shippuuden":500, "One Piece":784,
                "Crayon Shin chan":942,"Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25} 

In [67]:
for k,v in known_animes.items():    
    anime.loc[anime["name"]==k,"episodes"] = v

In [68]:
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)
anime["episodes"].fillna(anime["episodes"].median(),inplace = True)

### Data processing and Anime Rating 

In [69]:
anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].median(),inplace = True)

###Anime Category 

In [70]:
pd.get_dummies(anime[["type"]]).head()

,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [71]:
anime["members"] = anime["members"].astype(float)

##Feature Selection 

In [72]:
import re
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),pd.get_dummies(anime[["type"]]),anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)
anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))
anime_features.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,rating,members,episodes
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,9.37,200630.0,1
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,9.26,793665.0,64
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.25,114262.0,51
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.17,673572.0,24
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.16,151266.0,51


In [73]:
anime_features.columns

Index([' Adventure', ' Cars', ' Comedy', ' Dementia', ' Demons', ' Drama',
       ' Ecchi', ' Fantasy', ' Game', ' Harem', ' Hentai', ' Historical',
       ' Horror', ' Josei', ' Kids', ' Magic', ' Martial Arts', ' Mecha',
       ' Military', ' Music', ' Mystery', ' Parody', ' Police',
       ' Psychological', ' Romance', ' Samurai', ' School', ' Sci-Fi',
       ' Seinen', ' Shoujo', ' Shoujo Ai', ' Shounen', ' Shounen Ai',
       ' Slice of Life', ' Space', ' Sports', ' Super Power', ' Supernatural',
       ' Thriller', ' Vampire', ' Yaoi', ' Yuri', '1', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space',
       'Sports', 'Super Power', 'Supernat

###Data Scaling

In [74]:
max_abs_scaler = MaxAbsScaler()
anime_features = max_abs_scaler.fit_transform(anime_features)

###Establishing K Nearest Neighbours

In [75]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(anime_features)
distances, indices = nbrs.kneighbors(anime_features)

In [76]:
#cross_validate(nbrs, anime, measures=['RMSE', 'mae'], cv = 3)

###Key Query functions 

To Establish Movie index

In [77]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]
    

In [78]:
get_index_from_name("Noragami")

339

To search by a partial Name

In [79]:
all_anime_names = list(anime.name.values)
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [80]:
get_id_from_partial_name("Fairy")

Fairy Tail 2014  255
Fairy Tail 288
Fairy Tail OVA 796
Fairy Tail Movie 1 Houou no Miko 1053
Fairy Tail OVA 2016  1411
Fairy Tail x Rave 1445
Fairy Tail Movie 1 Houou no Miko Hajimari no Asa 1588
Wagamama Fairy Mirumo de Pon  1693
Ginga Ojousama Densetsu Yuna Shin 039 en no Fairy 5613
Fairy Tail Movie 2 10977
Fairy Forest Remi chan 12227


To find similar Anime Using model Prediction

In [81]:
def print_similar_animes(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.iloc[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(anime.iloc[id]["name"])

In [82]:
print_similar_animes(query="Fairy Tail")

Fairy Tail 2014 
Magi The Labyrinth of Magic
Magi The Kingdom of Magic
Densetsu no Yuusha no Densetsu
Magi Sinbad no Bouken TV 
